<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/LSTM/GeracaoAutomaticaDeTexto_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Geração Automática de Texto com LSTMs

As Redes Neurais Recorrentes também podem ser usadas como modelos generativos. Isso significa que além de serem usadas como modelos preditivos (fazendo previsões), elas podem aprender as sequências de um problema e, em seguida, gerar sequências plausíveis inteiramente novas para o domínio do problema. Modelos Generativos como este são úteis não apenas para estudar o quão bem um modelo aprendeu um problema, mas para saber mais sobre o próprio domínio do problema. 

Uma boa forma de praticar a criação de texto, é usando livros clássicos, os quais já temos uma boa ideia sobre a história e que não estejamos violando nenhum direito de copyright. Muitos livros clássicos já não possuem mais restrição de uso e podem ser usados gratuitamente. Um bom lugar para encontrar esses livros é no site do Projeto Gutenberg. É de lá que usaremos o livro para o qual criaremos um modelo generativo: Alice no País das Maravilhas ou o nome em inglês Alice's Adventures in Wonderland. O arquivo txt do livro pode ser baixado aqui: https://www.gutenberg.org/ebooks/11. 
Este livro tem cerca de 3.300 linhas de texto. O cabeçalho e a marca de final de arquivo foram removidos, já que não são necessários para o que vamos fazer.

Vamos aprender as dependências entre os caracteres e as probabilidades condicionais de caracteres em sequências para que possamos gerar sequências totalmente novas e originais de caracteres. Esta é uma tarefa divertida e recomendo repetir essas experiências com outros livros do Projeto Gutenberg. Essas experiências não se limitam ao texto, você também pode experimentar com outros dados ASCII, como código fonte de linguagens de programação, documentos marcados em LaTeX, HTML ou Markdown e muito mais. 

Faremos aqui algo muito similar ao que foi feito pelo programador, que escreveu um novo livro de Game ofthrones: http://www.businessinsider.com/ai-just-wrote-the-next-book-of-game-of-thrones-for-us-2017-8

In [0]:
# Imports
import numpy
import sys
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# Carregamos os dados e convertemos para lowercase 
# Estamos usando aqui arquivo texto no formato ASCII
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

Agora que o livro está carregado, devemos preparar os dados para modelagem. Não podemos modelar os caracteres diretamente, em vez disso, devemos converter os caracteres em números inteiros. Podemos fazer isso facilmente, criando um conjunto de todos os caracteres distintos do livro, então criando um mapa de cada caractere para um único inteiro.

In [0]:
# Criando o mapeamento caracter/inteiro
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
chars

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 ':',
 ';',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [0]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 '*': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 ':': 11,
 ';': 12,
 '?': 13,
 '[': 14,
 ']': 15,
 '_': 16,
 'a': 17,
 'b': 18,
 'c': 19,
 'd': 20,
 'e': 21,
 'f': 22,
 'g': 23,
 'h': 24,
 'i': 25,
 'j': 26,
 'k': 27,
 'l': 28,
 'm': 29,
 'n': 30,
 'o': 31,
 'p': 32,
 'q': 33,
 'r': 34,
 's': 35,
 't': 36,
 'u': 37,
 'v': 38,
 'w': 39,
 'x': 40,
 'y': 41,
 'z': 42,
 '\ufeff': 43}

Pode haver alguns caracteres que podemos remover para limpar mais o conjunto de dados que reduzirá o vocabulário e poderá melhorar o processo de modelagem. 

In [0]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: {:,d}".format(n_chars))  
print ("Total Vocab: ", n_vocab)

Total Characters: 144,343
Total Vocab:  44


Podemos ver que o livro tem pouco menos de 150.000 caracteres e que quando convertidos para minúsculas, existem apenas 44 caracteres distintos no vocabulário para a rede aprender, muito mais do que os 26 no alfabeto. Agora, precisamos definir os dados de treinamento para a rede. Existe muita flexibilidade em como você escolhe dividir o texto e expô-lo a rede durante o treino. Aqui dividiremos o texto do livro em subsequências com um comprimento de 100 caracteres, um comprimento arbitrário. Poderíamos facilmente dividir os dados por sentenças e ajustar as sequências mais curtas e truncar as mais longas. Cada padrão de treinamento da rede é composto de 100 passos de tempo (time steps) de um caractere (X) seguido por um caracter de saída (y). Ao criar essas sequências, deslizamos esta janela ao longo de todo o livro um caracter de cada vez, permitindo que cada caracter tenha a chance de ser aprendido a partir dos 100 caracteres que o precederam (exceto os primeiros 100 caracteres, é claro). Por exemplo, se o comprimento da sequência é 5 (para simplificar), os dois primeiros padrões de treinamento seriam os seguintes:

* Palavra: CHAPTER
* CHAPT -> E
* HAPTE -> R

In [0]:
# À medida que dividimos o livro em sequências, convertemos os caracteres em números inteiros usando nossa
# tabela de pesquisa que preparamos anteriormente.
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total de Padrões: ", n_patterns)

Total de Padrões:  144243


Agora que preparamos nossos dados de treinamento, precisamos transformá-lo para que possamos usá-lo com o Keras. Primeiro, devemos transformar a lista de sequências de entrada na forma [amostras, passos de tempo, recursos] esperados por uma rede LSTM. Em seguida, precisamos redimensionar os números inteiros para o intervalo de 0 a 1 para tornar os padrões mais fáceis de aprender pela rede LSTM que usa a função de ativação sigmoide por padrão.

In [0]:
# Reshape de X para [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Normalização
X = X / float(n_vocab)

Finalmente, precisamos converter os padrões de saída (caracteres únicos convertidos em números inteiros) usando Hot-Encoding. Isto é para que possamos configurar a rede para prever a probabilidade de cada um dos 44 caracteres diferentes no vocabulário (uma representação mais fácil) em vez de tentar forçá-lo a prever com precisão o próximo caracter. Cada valor de y é convertido em um vetor com um comprimento 44, cheio de zeros, exceto com um 1 na coluna para a letra (inteiro) que o padrão representa. Por exemplo, quando a letra n (valor inteiro 30) tiver sido transformada usando One-Hot Encoding, vai se parecer com isso:

[ 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [0]:
# One-Hot Encoding da variável de saída
y = np_utils.to_categorical(dataY)

In [0]:
# Modelo LSTM com duas camadas de Dropout com 20%
# O tempo de treinamento é bem longo
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Não há conjunto de dados de teste. Estamos modelando todo o conjunto de dados de treinamento para aprender a probabilidade de cada caracter em uma sequência. Não estamos interessados nos mais preciso modelo do conjunto de dados de treinamento (Acurácia de Classificação). Este seria um modelo que prevê cada caracter no conjunto de dados de treinamento perfeitamente. Em vez disso, estamos interessados em uma generalização do conjunto de dados que minimiza a função de perda escolhida. Estamos buscando um equilíbrio entre generalização e
overfitting.

In [0]:
# Define o checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5-English"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

Fit do modelo

In [0]:
# model.fit(X, y, epochs = 20, batch_size = 128, callbacks = callbacks_list)
model.fit(X, y, epochs = 50, batch_size = 64, callbacks = callbacks_list)

Depois de executar o fit, você deve ter uma série de arquivos de checkpoint no mesmo diretório onde está este Jupyter Notebook. Você pode excluí-los todos exceto aquele com o menor valor de perda. Por exemplo, neste caso, o arquivo weights-improvement-19-1.9119.hdf5 será usado. Ele contém os melhores valores de peso.

In [0]:
# Carrega os melhores pesos da rede e compila o modelo
filename = "weights-improvement-49-1.3344.hdf5-English"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
def generate_text():
    # Obtém um random seed
    start = numpy.random.randint(0, len(dataX)-1)

    # Inicia a geração de texto de um ponto qualquer, definido pelo random seed "start"
    pattern = dataX[start]
    print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

    # Gerando caracteres
    for i in range(1000):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        index = numpy.argmax(prediction)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        sys.stdout.write(result)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
    print('\n=================================================================================' +
          '================================\n')     
    #print ("\nConcluído.")

In [0]:
for i in range(4):
    generate_text()

" ir tails in their mouths; and the
reason is--' here the mock turtle yawned and shut his eyes.--'tell "
 as the cookows with the same way the way in the pight with the same as the court, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with with the soof, and the soom aadk again, and she was the pueen said to herself 'it was a parhe care with the same with w

In [0]:
for i in range(5):
    generate_text()

" shion, and
this was her dream:--

first, she dreamed of little alice herself, and once again the tin "
e she had not the poor of the garden, and was a little shaten taie to her eyes, and the words were all suite sueeen the cook with the soom of the door, and the soom aadk again, and she was a very curious senarking to herself 'it was a parhe mittle birds with the same with what it was a large canlessations and the soofs, and the soom as she was a very curious semark, and the doom had to say the caby with the soom of the door and the tooes, and she was not and sooetheng about the caby was the pueen said to her eyes, and the soom aadk again, and she was a very curious cropiens and the caby was snon and seemed to be a lettor of tee it was a great duactle the caby was the pueen sat again, and the soom of the door and the soo of her head to be tee with the same with with the sooes, and the soom of the door and the soo of her head to be a little book with the soom of the door, and the soom